In [1]:
import numpy as np
import pandas as pd

# constants
a = 0.16 # (N5-N3)/N5 measured from steady-state Z5 and Z3 levels
b = 0.02 # kd1, measured from rif experiment. 1.28/60 (in per sec)
t = 75 # tau or transcription time/2 (in sec)
l = 150 # 1/kd2 of Z3 (in sec)

#x = k_d1* (in per sec), y = k_dPT (in per sec), z = PT (number between 0 and 1)

# intermediaries
def u(x):
    return np.exp(-x * t)

A = a - 1
L = lambda x: A * (1 - u(x)**2) + l * x
M = lambda x: A * u(x) * (1-u(x)) + l * x
N = lambda x: A * x * u(x)

K = lambda x: (M(x) * x - N(x) * u(x)) / L(x) / u(x)
J = lambda x: - M(x) * (x - b) / L(x) / u(x)

P = lambda x: u(x) * K(x)
Q = lambda x: u(x) * J(x) - x
R = lambda x: x - b

# discriminant
D = lambda x: Q(x)**2 - 4 * P(x) * R(x)

# solutions
z1 = lambda x: (-Q(x) + np.sqrt(D(x))) / 2 / P(x)
z2 = lambda x: (-Q(x) - np.sqrt(D(x))) / 2 / P(x)

y1 = lambda x: K(x) * z1(x) + J(x)
y2 = lambda x: K(x) * z2(x) + J(x)

# run this to check solutions per x 
def check_solutions(x, eps=1e-12):
    check_solution(x, y1(x), z1(x), eps=eps)
    check_solution(x, y2(x), z2(x), eps=eps)
   
def check_solution(x, y, z, eps=1e-12):
    v = u(x)
    assert np.abs(x * (1-z) + y * z * u(x) - b) < eps
    assert (z - A * y * (1-v**2) + l * x * y) / ( A * y * v * (1-v) - A * x * v + l * x * y) - z

In [5]:
# table
grid = np.arange(0.005723, 0.01, 0.001245/2) #for x
# based on BCM experiment, we consider x = 0.38 /min = 0.006345 /s
pd.DataFrame([grid, [y1(x) for x in grid], [z1(x) for x in grid], [y2(x) for x in grid], [z2(x) for x in grid]], index=['x', 'y1', 'z1', 'y2', 'z2']).T


,x,y1,z1,y2,z2
0,0.005723,0.051452,0.514060,-0.003562,-1.775278
1,0.006345,0.051463,0.532771,-0.003243,-1.633228
2,0.006968,0.051687,0.550304,-0.002948,-1.495146
3,0.007590,0.052102,0.566766,-0.002676,-1.362986
4,0.008213,0.052687,0.582251,-0.002423,-1.237913
5,0.008835,0.053429,0.596840,-0.002189,-1.120506
6,0.009458,0.054319,0.610602,-0.001972,-1.010931


In [3]:
df = pd.DataFrame([grid, [y1(x) for x in grid], [z1(x) for x in grid], [y2(x) for x in grid], [z2(x) for x in grid]], index=['x', 'y1', 'z1', 'y2', 'z2']).T
df.to_csv("cc2.csv")